<a href="https://colab.research.google.com/github/GN-Yu/TSRL-project/blob/main/one_dimension.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import psutil
import subprocess
import random
import numpy as np
import tensorflow as tf

# Function to execute shell command and return output
def run_shell_command(cmd):
  return subprocess.run(cmd, stdout=subprocess.PIPE, shell=True).stdout.decode('utf-8').strip()

# Check CPU
cpu_info = run_shell_command("cat /proc/cpuinfo | grep 'model name' | uniq")
print("CPU:", cpu_info.split(":")[1].strip() if cpu_info else "Not available")

# Number of CPU Cores
num_cores = os.cpu_count()
print("Number of CPU cores:", num_cores)

# Total RAM
ram_info = psutil.virtual_memory()
total_ram = ram_info.total / (1024 ** 3)  # Convert bytes to GB
print(f"Total RAM : {total_ram} GB")

# Check GPU
gpu_info = !nvidia-smi --query-gpu=gpu_name,memory.total --format=csv
if gpu_info and len(gpu_info) > 1:
  gpu_name = gpu_info[1].split(',')[0]
  gpu_memory_mib = float(gpu_info[1].split()[2])  # Extract the memory in MiB
  gpu_memory_gb = gpu_memory_mib / 1024
  print(f"GPU: {gpu_name}, {gpu_memory_gb} GB")
else:
  print("GPU: Not available")

# TensorFlow info
print(f"TensorFlow version: {tf.__version__}")
print("TensorFlow GPU Availability:", tf.test.is_gpu_available())

CPU: Intel(R) Xeon(R) CPU @ 2.00GHz
Number of CPU cores: 2
Total RAM : 12.674789428710938 GB


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


GPU: Tesla T4, 15.0 GB
TensorFlow version: 2.15.0
TensorFlow GPU Availability: True


In [2]:
# organize files
for dirname, _, filenames in os.walk('/content/drive/MyDrive/TSRL_project'):
  for filename in filenames:
    print(os.path.join(dirname, filename))

In [3]:
def sample_brownian_motion(num_trajectories, time_steps):
  """
  Function to sample Brownian motion for the specified number of trajectories and time steps.

  :param num_trajectories: The number of trajectories to simulate.
  :param time_steps: A numpy array of time steps.
  :return: A numpy array of shape (num_trajectories, len(time_steps)-1) representing the Brownian increments.
  """
  # Calculate time intervals (delta_t) as differences between consecutive time steps
  delta_t = np.diff(time_steps)

  # Sample Brownian increments ∆W for each time interval and each trajectory
  brownian_increments = np.random.normal(0, np.sqrt(delta_t), (num_trajectories, len(delta_t)))

  return brownian_increments

def sample_initial_states(num_samples):
  # Implement initial state sampling here
  pass



In [4]:
# @title Levy jump func in tuple format

# def sample_levy_jumps(num_trajectories, lambda_param, mu, sigma, time_horizon):
#     """
#     Function to sample jumps from a Lévy process for the specified number of trajectories.

#     :param num_trajectories: The number of trajectories to simulate.
#     :param lambda_param: The intensity parameter λ of the Poisson process.
#     :param phi_function: The density function ϕ(z) of the jump sizes.
#     :param time_horizon: The total time horizon T.
#     :return: A list containing tuples (jump_times, jump_sizes) for each trajectory.
#     """
#     jumps = []

#     for _ in range(num_trajectories):
#         # Step (a): Generate a sequence of exponential distributions with parameter λ
#         exponential_samples = np.random.exponential(1/lambda_param, 1000)  # Generate a large number of samples

#         # Step (b): Compute the cumulative sum to get the arrival times of the Poisson process
#         arrival_times = np.cumsum(exponential_samples)
#         arrival_times = arrival_times[arrival_times <= time_horizon]  # Filter times beyond the time horizon

#         # Step (c): Sample from ϕ(z) for each arrival time to get the jump sizes
#         jump_sizes = np.array([np.random.normal(mu, sigma) for _ in arrival_times])

#         jumps.append((arrival_times, jump_sizes))

#     return jumps

# # Example usage
# lambda_param = 0.3  # Intensity parameter λ of the Poisson process
# time_horizon = 1    # Total time horizon T
# M = 1000  # Number of samples/trajectories
# mu = 0.4
# sigma = 0.25

# np.random.seed(SEED)

# # Simulate jumps for a specific number of trajectories
# levy_jumps = sample_levy_jumps(M, lambda_param, mu, sigma, time_horizon)

# # Example output of the first trajectory's jumps
# for i in range(10):
#   print(levy_jumps[i])  # (array of arrival times, array of jump sizes)

# print(len(levy_jumps))

In [5]:
def sample_levy_jumps(num_trajectories, lambda_param, mu, sigma, time_horizon, max_jumps=1000):
  """
  Function to sample jumps from a Lévy process for the specified number of trajectories.
  Returns a 3D NumPy array with shape (num_trajectories, max_jumps, 2), where the last dimension
  contains arrival times and jump sizes respectively.

  :param num_trajectories: The number of trajectories to simulate.
  :param lambda_param: The intensity parameter λ of the Poisson process.
  :param mu: Mean of the normal distribution for jump sizes.
  :param sigma: Standard deviation of the normal distribution for jump sizes.
  :param time_horizon: The total time horizon T.
  :param max_jumps: Maximum number of jumps to consider for each trajectory.
  :return: A 3D NumPy array containing (arrival_times, jump_sizes) for each trajectory.
  """
  all_jumps = np.zeros((num_trajectories, max_jumps, 2))

  for i in range(num_trajectories):
    # Generate a sequence of exponential distributions with parameter λ
    exponential_samples = np.random.exponential(1/lambda_param, max_jumps)

    # Compute the cumulative sum to get the arrival times of the Poisson process
    arrival_times = np.cumsum(exponential_samples)
    valid_indices = arrival_times <= time_horizon
    arrival_times = arrival_times[valid_indices]

    # Sample from the normal distribution for each arrival time to get the jump sizes
    jump_sizes = np.random.normal(mu, sigma, arrival_times.size)

    all_jumps[i, :arrival_times.size, 0] = arrival_times
    all_jumps[i, :arrival_times.size, 1] = jump_sizes

  return all_jumps


# # Example usage
# lambda_param = 0.3  # Intensity parameter λ of the Poisson process
# time_horizon = 1    # Total time horizon T
# M = 1000  # Number of samples/trajectories
# mu = 0.4
# sigma = 0.25

# SEED = 2023
# np.random.seed(SEED)

# # Simulate jumps for a specific number of trajectories
# levy_jumps = sample_levy_jumps(M, lambda_param, mu, sigma, time_horizon)

# # Example output of the first trajectory's jumps
# for i in range(10):
#   print(levy_jumps[i])  # (array of arrival times, array of jump sizes)

# print(len(levy_jumps))

In [6]:
# from scipy.stats import norm

# z = 0.5 # location to evaluate

# # Using scipy.stats
# phi = norm.pdf(z, loc=mu, scale=sigma)

# print(phi)

In [7]:
def create_neural_network(input_dim, layer_width=25, output_dim=2):
  # Define the input layer
  inputs = tf.keras.Input(shape=(input_dim,))

  # First linear layer
  x = tf.keras.layers.Dense(layer_width, activation='tanh')(inputs)

  # Add five residual blocks
  for _ in range(5):
      y = tf.keras.layers.Dense(layer_width, activation='tanh')(x)
      y = tf.keras.layers.Dense(layer_width, activation='tanh')(y)
      x = tf.keras.layers.Add()([x, y])

  # Final linear layer
  outputs = tf.keras.layers.Dense(output_dim)(x)

  model = tf.keras.Model(inputs=inputs, outputs=outputs)
  return model

def create_simplified_model(input_dim, output_dim=2):
    # Define the input layer
    inputs = tf.keras.Input(shape=(input_dim,))

    # A single dense layer
    outputs = tf.keras.layers.Dense(output_dim)(inputs)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

# # Use GPU for model creation
# with tf.device('/GPU:0'):
#     model = create_neural_network(input_dim=2)

# # Prepare input data (example)
# input_data = np.array([[0.5, -0.5],
#                        [1.0, 0.0],
#                        [-0.5, 0.5]])

# # Make predictions
# output_data = model.predict(input_data)

# print("Output Data:")
# print(output_data)

In [8]:
import scipy.integrate as integrate
from scipy.stats import norm

def G_function(x, z):
  """Define the function G(x, z)."""
  return x * np.exp(z)

def normal_pdf(z, mu, sigma):
  """Normal probability density function."""
  return (1 / (np.sqrt(2 * np.pi) * sigma)) * np.exp(-((z - mu) ** 2) / (2 * sigma ** 2))

def integral_G_phi(x, mu, sigma, G_func, phi_func):
  """Calculate the integral of G_func(x, z) * phi_func(z) over R."""
  def integrand(z):
      return G_func(x, z) * phi_func(z, mu, sigma)

  # Perform the integration over the entire real line
  result, _ = integrate.quad(integrand, -500, 500)
  return result

def g_function(x):
  return x

# # Example usage:
# mu = 0.4
# sigma = 0.25
# x = 1  # Example value for x
# integral = integral_G_phi(x, mu, sigma, G_function, normal_pdf)
# print(integral)


In [9]:
#@title non-vectorized calcs

# np.random.seed(SEED)

# # Define Constants

# lambda_poisson = 0.3
# mu = 0.4
# sigma = 0.25

# T = 1.0  # Total time
# N = 50  # Number of time intervals
# # M = 1000  # Number of samples/trajectories
# # Iterations = 400  # Number of iterations

# # unit tests
# M = 5  # Number of samples/trajectories
# Iterations = 1  # Number of iterations

# initial_learning_rate = 5e-5

# # Generate equally spaced time snapshots from 0 to T
# time_steps = np.linspace(0, T, N+1)

# # Initialize NN model
# d = 1 # problem's dimensionality
# input_dim = d + 1
# with tf.device('/GPU:0'):
#   model = create_neural_network(input_dim)


# for iteration in range(Iterations):
#   brownian_motions = sample_brownian_motion(M, time_steps)
#   levy_jumps = sample_levy_jumps(M, lambda_poisson, mu, sigma, T)

#   X = [[0 for _ in range(M)] for _ in range(N+1)]
#   Y = [[0 for _ in range(M)] for _ in range(N+1)]

#   for timestep, time in enumerate(time_steps):
#     if timestep == 0:
#       X[timestep] = [1 for _ in range(M)]  # Initial value for PIDE
#     else:
#       timestep_pre = timestep - 1
#       time_pre = time_steps[timestep_pre]
#       time_diff = time - time_pre

#       for trajectory in range(M):
#         arrival_times, jump_sizes = levy_jumps[trajectory]
#         mask = (arrival_times > time_pre) & (arrival_times <= time)

#         X[timestep][trajectory] = X[timestep_pre][trajectory] + G_function(X[timestep_pre][trajectory], jump_sizes[mask]).sum() - time_diff * lambda_poisson * integral_G_phi(X[timestep_pre][trajectory], mu, sigma, G_function, normal_pdf)



In [44]:
# Set random seed for reproducibility
SEED = 2023
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)

# Constants
lambda_poisson = 0.3
mu = 0.4
sigma = 0.25
T = 1.0  # Total time
N = 50   # Number of time intervals
# M = 1000  # Number of samples/trajectories
# Iterations = 400  # Number of iterations

# unit tests
M = 5  # Number of samples/trajectories
Iterations = 2  # Number of iterations

initial_learning_rate = 5e-5


# Generate equally spaced time snapshots from 0 to T
time = np.linspace(0, T, N+1)

# Initialize NN model
d = 1  # problem's dimensionality
input_dim = d + 1
with tf.device('/GPU:0'):
  model = create_neural_network(input_dim)

# Optimizer
optimizer = tf.keras.optimizers.Adam()

# Ensure the model is set to training mode
model.trainable = True

# Randomly generate terminal state (normal with mean of intial value)
X_terminal = np.random.normal(1, 0.1, M)
# Convert X_terminal to a TensorFlow Tensor
X_terminal_tensor = tf.convert_to_tensor(X_terminal)

for iteration in range(Iterations):
  with tf.GradientTape(persistent=True) as tape:
    brownian_motions = sample_brownian_motion(M, time)
    levy_jumps = sample_levy_jumps(M, lambda_poisson, mu, sigma, T)

    # Initialize arrays for X and Y
    X = np.zeros((N+1, M))
    Y = np.zeros((N+1, M))

    # Set initial and terminal value for X
    X[0, :] = 1
    X[N, :] = X_terminal

    # Iterate over time steps
    for timestep in range(N):
      time_diff = time[timestep + 1] - time[timestep]

      # Mask for non-zero jumps that occur in the current time interval
      masks = (levy_jumps[:, :, 0] > time[timestep]) & (levy_jumps[:, :, 0] <= time[timestep + 1])

      # Vectorized computation of jump contributions for each trajectory
      X_current = X[timestep, :][:, np.newaxis]  # Reshape X to for broadcasting
      jump_contributions = np.sum(G_function(X_current, levy_jumps[:, :, 1]) * masks, axis=1)

      # Apply integral_G_phi element-wise
      integral_contributions = np.array([time_diff * lambda_poisson * integral_G_phi(x, mu, sigma, G_function, normal_pdf) for x in X[timestep, :]])

      # Calculate X for next time step
      X[timestep + 1, :] = X[timestep, :] + jump_contributions - integral_contributions

      NN_outputs_current = model(np.stack([np.full(M, time[timestep]), X[timestep, :]], axis=1)) # can be optimized for calc

      NN_outputs_next = model(np.stack([np.full(M, time[timestep+1]), X[timestep+1, :]], axis=1)) # can be optimized for calc

      TD_error_jump_contribution = np.zeros(M)

      for trajectory in range(M):
        # Filter levy_jumps for the current trajectory and time interval
        relevant_jumps = [jump for jump in levy_jumps[trajectory] if time[timestep] < jump[0] <= time[timestep + 1]]

        # If there are no relevant jumps, set TD_error_jump_contribution for this trajectory to 0
        if not relevant_jumps:
          TD_error_jump_contribution[trajectory] = 0
        else:
          TD_error_jump_contribution[trajectory] = sum(
              model(np.array([[time[timestep], X[timestep, trajectory] + G_function(X[timestep, trajectory], jump[1])]]))[0,0]
              - model(np.array([[time[timestep], X[timestep, trajectory]]]))[0,0]
              for jump in relevant_jumps
          )

      # Loss 1: TD error
      TD_error = tf.reduce_mean(tf.square(TD_error_jump_contribution - time_diff * NN_outputs_current[:, 1] + NN_outputs_current[:, 0] - NN_outputs_next[:, 0]))

      # Loss 4: N1 and N2 constraint error
      constraint_error = tf.abs(tf.reduce_mean(TD_error_jump_contribution - time_diff * NN_outputs_current[:, 1]))

      if timestep == N-1:
        X_terminal_tensor = tf.convert_to_tensor(X[N, :])

      terminal_input = tf.convert_to_tensor(np.stack([np.full(M, T), X_terminal_tensor], axis=1))

      tape.watch(terminal_input)  # Ensure terminal_input is being watched
      tape.watch(X_terminal_tensor)  # Ensure X_terminal_tensor is being watched

      NN_outputs_terminal = model(terminal_input)

      N1_gradient_terminal = tape.gradient(NN_outputs_terminal[:, 0], terminal_input)[:, 1]

      g_X_terminal = tf.cast(g_function(X_terminal_tensor), tf.float32)
      g_gradient = tape.gradient(g_X_terminal, X_terminal_tensor)

      # Loss 2: Y terminal value error
      Y_terminal_error = tf.reduce_mean(tf.square(NN_outputs_terminal[:, 0] - g_X_terminal)) / N

      # Loss 3: Y terminal value derivative error
      Y_teriminal_deriv_error = tf.cast(tf.reduce_mean(tf.square(N1_gradient_terminal - g_gradient)) / N, tf.float32)

      loss = TD_error + Y_terminal_error + Y_teriminal_deriv_error + constraint_error

      print(loss)


tf.Tensor(0.00797916, shape=(), dtype=float32)
tf.Tensor(0.0076852306, shape=(), dtype=float32)
tf.Tensor(0.007381106, shape=(), dtype=float32)
tf.Tensor(0.007067344, shape=(), dtype=float32)
tf.Tensor(0.0067445654, shape=(), dtype=float32)
tf.Tensor(0.006413544, shape=(), dtype=float32)
tf.Tensor(0.0060751257, shape=(), dtype=float32)
tf.Tensor(0.0057302844, shape=(), dtype=float32)
tf.Tensor(0.005623257, shape=(), dtype=float32)
tf.Tensor(0.0059728855, shape=(), dtype=float32)
tf.Tensor(0.0063265455, shape=(), dtype=float32)
tf.Tensor(0.0066829687, shape=(), dtype=float32)
tf.Tensor(0.007040894, shape=(), dtype=float32)
tf.Tensor(0.0073990664, shape=(), dtype=float32)
tf.Tensor(0.0077563003, shape=(), dtype=float32)
tf.Tensor(0.008111456, shape=(), dtype=float32)
tf.Tensor(0.008463527, shape=(), dtype=float32)
tf.Tensor(0.00881158, shape=(), dtype=float32)
tf.Tensor(0.009154849, shape=(), dtype=float32)
tf.Tensor(0.0094927, shape=(), dtype=float32)
tf.Tensor(0.009824661, shape=(), dt

In [ ]:
# Define your multi-dimensional function
def my_custom_function(x, t):
    # Example computation; replace this with your actual function
    return t * x

# Example multi-dimensional input
# Let's assume the input has shape [batch_size, input_dim]
# For example, batch_size = 3, input_dim = 2
x = tf.Variable([[1.0, 2.0], [1.0, 2.0], [1.0, 2.0]], dtype=tf.float32)

with tf.GradientTape() as tape:
    tape.watch(x)  # Ensure x is being watched for gradient computation
    output = my_custom_function(x, 2)

# Compute the gradient of the output with respect to the input
grads = tape.gradient(output, x)

print(grads)  # This will print the gradient tensor


In [ ]:
simplified_input = tf.constant([[T, 1.0]], dtype=tf.float32)
with tf.GradientTape() as tape:
    tape.watch(simplified_input)
    simplified_output = model(simplified_input)
    simplified_gradient = tape.gradient(simplified_output, simplified_input)
print("Simplified Gradient:", simplified_gradient)


In [ ]:
import tensorflow as tf

# Example neural network with multiple outputs
model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(2)  # Let's say we have 2 outputs
])

# Example input
x = tf.Variable([[0.1], [0.2], [0.3]], dtype=tf.float32)

with tf.GradientTape(persistent=True) as tape:
    tape.watch(x)
    outputs = model(x)

# Assuming outputs is a tensor of shape [batch_size, num_outputs]
gradients = []
for i in range(outputs.shape[1]):  # Loop over each output
    # Compute the gradient of the i-th output with respect to x
    grad = tape.gradient(outputs[:, i], x)
    gradients.append(grad)

# gradients now contains the gradients of each output with respect to x
